# Predicting Star Ratings of Edinburgh Airbnbs through Review Texts Analysis

# Notebook 1: Data Cleaning

In this notebook, we will first outline the objectives of the project. Then, we will begin the foundational steps, including loading the datasets, analyzing the contents of our data, and performing the first stage of the data cleaning process.

***

# Table of Contents

# Introduction

#### Problem Statement

For this analysis, the focus lies in the quest to improve the hosting experience on Airbnb, particularly in the city of Edinburgh. One challenge Airbnb hosts face is understanding what factors contribute to 
their Airbnb listing's star ratings, which significantly impact their bookings and revenue.

We aim to uncover valuable insights by analyzing review texts. Our primary goal is to predict average star ratings based on these reviews, providing hosts with actionable feedback on areas of improvement. By extracting key features from reviews, such as cleanliness, communication, and amenities, we can offer guidance to both new hosts seeking to establish themselves and experienced hosts looking to enhance their offerings. 

Ultimately, this analysis aims to empower hosts by providing useful insights and actionable recommendations to enhance their performance and maximize their ratings. Furthermore, we aim for these insights to not only benefit hosts in Edinburgh but also serve as valuable guidance for hosts in other cities facing similar challenges in the competitive Airbnb landscape.

#### Data Collection

The data used in this project consists of two datasets downloaded from [Inside Airbnb](http://insideairbnb.com/get-the-data). The first dataset contains information about Edinburgh Airbnb Listings, and the second dataset contains the reviews of these listings.

The data dictionary can be downloaded [here](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=1322284596).

***

# Data descriptions

## Import libraries

In [1]:
# Main libraries
import pandas as pd
import numpy as np

# Library for extracting data
import joblib

***

## Dataset descriptions

In [2]:
# Import datasets
df_listings= pd.read_csv('data/listings.csv')
df_reviews=pd.read_csv('data/reviews.csv')

In [3]:
# Show the first row of the listing dataframe
# As we have too many columns, we will make a matrix transpose to show all information
df_listings.head(1).T

,0
id,15420
listing_url,https://www.airbnb.com/rooms/15420
scrape_id,20231217045056
last_scraped,2023-12-17
source,city scrape
...,...
calculated_host_listings_count,1
calculated_host_listings_count_entire_homes,1
calculated_host_listings_count_private_rooms,0
calculated_host_listings_count_shared_rooms,0


In [4]:
# Show the first row of the reviews dataframe
df_reviews.head(1)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,15420,171793,2011-01-18,186358,Nels,My wife and I stayed at this beautiful apartme...


In [5]:
# Show the dimensions of the datasets.
print(f'The shape of the listing dataset is {df_listings.shape}')
print(f'The shape of the review dataset is {df_reviews.shape}')

The shape of the listing dataset is (7049, 75)
The shape of the review dataset is (535577, 6)


We have two datasets that need to perform data cleaning at this stage. The Airbnb listing dataset contains **7049** rows and **75** columns. The reviews dataset contains **535577** rows and **6** columns. We will analyse the columns for these two datasets in more detail later, and eventually we will merge the information into one dataframe.

In [6]:
# Show the list that contains listings dataframe's columns
print(f'The initial columns in the listing dataset are')
list(df_listings.columns)

The initial columns in the listing dataset are


['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [7]:
# Show the list that contains review dataframe's columns
print(f'The initial columns in the review dataset are')
list(df_reviews.columns)

The initial columns in the review dataset are


['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

## Data Dictionaries of Airbnb listing dataset

In [8]:
# Import raw data dictionary csv
df_dictionary = pd.read_csv('data/Data Dictionary.csv')

# Drop the first 6 unnecessary rows
df_dictionary_clean=df_dictionary.drop(range(6), axis=0)

# Reset index
df_dictionary_clean.reset_index(drop=True, inplace=True)

# Use the first row as column rows
df_dictionary_clean.columns=list(df_dictionary_clean.loc[0])

# Remove the first row
df_dictionary_clean.drop(0, inplace=True, axis=0)

# Reset index
df_dictionary_clean.reset_index(inplace=True, drop = True)

# Drop the 'Calculated' and 'Reference' column
df_dictionary_clean.drop(['Calculated', 'Reference'], axis=1, inplace=True)

In [9]:
# Show the data dictionary dataframe
df_dictionary_clean

,Field,Type,Description
0,id,integer,Airbnb's unique identifier for the listing
1,listing_url,text,NaN
2,scrape_id,bigint,"Inside Airbnb ""Scrape"" this was part of"
3,last_scraped,datetime,"UTC. The date and time this listing was ""scrap..."
4,source,text,"One of ""neighbourhood search"" or ""previous scr..."
...,...,...,...
74,reviews_per_month,numeric,The number of reviews the listing has over the...
75,NaN,NaN,NaN
76,Change control,NaN,NaN
77,Field,Change,NaN


## Rating types overview

**review_scores_rating**	
- Overall experience. Overall, how was the stay?

**review_scores_cleanliness**
- Cleanliness. Did guests feel that the space was clean and tidy?

**review_scores_accuracy**
- Accuracy. How accurately did the listing page represent the space? For example, guests should be able to find up-to-date info and photos in the listing description.

**review_scores_value**
- Value. Did the guest feel that the listing provided good value for the price?

**review_scores_communication**
- Communication. How well did you communicate before and during the stay? Guests often care that their host responds quickly, reliably, and frequently to their messages and questions.

**review_score_checkin**
- Check-in. How smoothly did check-in go?

**review_score_location**
- Location. How did guests feel about the neighbourhood? This may mean that there’s an accurate description for proximity and access to transportation, shopping centres, city centre, etc., and a description that includes special considerations, like noise, and family safety.

***

## Dataset summary overview

In [10]:
# Check the listing dataframe data types
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7049 non-null   int64  
 1   listing_url                                   7049 non-null   object 
 2   scrape_id                                     7049 non-null   int64  
 3   last_scraped                                  7049 non-null   object 
 4   source                                        7049 non-null   object 
 5   name                                          7049 non-null   object 
 6   description                                   0 non-null      float64
 7   neighborhood_overview                         4813 non-null   object 
 8   picture_url                                   7049 non-null   object 
 9   host_id                                       7049 non-null   i

In [11]:
# Check the percentage of null values in listing dataset
pd.set_option('display.max_rows', 75)
print("Initial Percentage of Null Values in the Listing dataset %")
print(round(df_listings.isna().sum()/df_listings.shape[0]*100, 2).sort_values (ascending=False)) #coverted into a percent(%) and round to 2 decimal places

Initial Percentage of Null Values in the Listing dataset %
bedrooms                                        100.00
description                                     100.00
calendar_updated                                100.00
bathrooms                                       100.00
neighbourhood_group_cleansed                    100.00
license                                          90.17
host_neighbourhood                               63.85
host_about                                       43.59
neighbourhood                                    31.72
neighborhood_overview                            31.72
host_response_rate                               26.54
host_response_time                               26.54
price                                            21.39
has_availability                                 21.39
host_location                                    17.49
host_acceptance_rate                             12.46
first_review                                      8.87
last_r

In [12]:
# Check the review dataframe data types
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535577 entries, 0 to 535576
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     535577 non-null  int64 
 1   id             535577 non-null  int64 
 2   date           535577 non-null  object
 3   reviewer_id    535577 non-null  int64 
 4   reviewer_name  535577 non-null  object
 5   comments       535542 non-null  object
dtypes: int64(3), object(3)
memory usage: 24.5+ MB


In [13]:
# Check the percentage of null values in review dataset
print("Initial Percentage of Null Values in the Review dataset %")
print(round(df_reviews.isna().sum()/df_reviews.shape[0]*100, 2).sort_values (ascending=False)) #coverted into a percent(%) and round to 2 decimal places

Initial Percentage of Null Values in the Review dataset %
comments         0.01
listing_id       0.00
id               0.00
date             0.00
reviewer_id      0.00
reviewer_name    0.00
dtype: float64


The listing dataset contains multiple columns with null values. We will investigate all of the columns by category first considering we have over 70 columns in the listing dataset. Then we will drop the columns if they either contain no useful information, contain only null values, or contain replicated information that we can retrieve from the other columns. We will also investigate the rows that contain missing values from the remaining columns and fillin these values if necessary before we decide to drop these rows. 

The review dataset seems quite clean, as we only have `comments` column contains less than 0.1% of null values, we will drop these rows from the review dataset.

***

## Clean the Review Dataset

First, we will clean the review dataset by removing the rows with no reviews and alson remove the rows which corresponds to listings with less than 3 reviews. This is so that our review analysis later will have enough reviews for each of the Airbnb listing.

First, we will extract the listing ids corresponds to the rows with no reviews in the review dataset, and check if these listings have no reviews at all or just these rows.

In [14]:
# Create a dataframe contains these null review data
null_reviews = df_reviews[df_reviews['comments'].isnull()]

# Check if the listings in the null_reviews dataframe have no reviews at all in the entire dataframe
df_reviews.groupby('listing_id').count().loc[null_reviews['listing_id']].isnull().sum()

id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

Therefore, we can use all the unique listing ids in the reviews dataframe to filter our listing dataframe after we drop these rows by extracting the listing ids that have more than 3 reviews.

In [15]:
# Remove these rows in the reviews dataframe
df_reviews_clean=df_reviews.drop(null_reviews.index, axis=0)

# Reset index
df_reviews_clean=df_reviews_clean.reset_index(drop=True)

# Filter listing data and only select listings with more than 3 reviews
review_counts=df_reviews_clean.groupby('listing_id').count()['id']

# Generate listing ids which has more than 3 reviews
filtered_list=[i for i in review_counts.index if review_counts[i]>=3]

# Select 'listing_id' column as dataframe index
df_reviews_clean.set_index('listing_id', inplace=True)

# Remove rows that have ids not in the generated reviews filtered list and reset index.
df_reviews_clean=df_reviews_clean.loc[filtered_list].reset_index()

***

## Clean the Listing Dataset

First, we will remove the listings from the extracted listing ids that have less than 3 reviews.

### Remove listings with less than 3 reviews

In [16]:
# Select 'listing_id' column as dataframe index
df_listings_clean=df_listings.set_index('id')

# Remove rows that have ids not in the generated reviews filtered list and reset index.
df_listings_clean=df_listings_clean.loc[filtered_list].reset_index()

Before we remove the columns that only contain null values, we will first remove the urls and scraping related columns.

### Remove url/Scraping columns

In [17]:
# Remove url and scrape related columns
ls=[]
for i in df_listings_clean.columns:
    if ('url' in i) | ('scrape' in i):
        ls.append(i)
df_listings_clean.drop(ls, axis=1, inplace=True)
df_listings_clean.drop('source', axis=1, inplace=True)

### Remove columns only contain null values

In [18]:
# Remove columns contain no values
for i in df_listings_clean.columns:
    if df_listings_clean.isnull().sum()[i] == df_listings_clean.shape[0]:
        df_listings_clean.drop(i, axis=1, inplace=True)    

### Clean Host related columns

First, we will drop the columns that contain replicated information.

In [19]:
# Create a list which has host related column names
host_columns=[]
for i in df_listings_clean.columns:
    if 'host' in i:
        host_columns.append(i)
# Show the host related column names
print(f'The columns that are related to the listing hosts are')
host_columns

The columns that are related to the listing hosts are


['host_id',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

We will remove the `host_name` column as names are irrelevant for our analysis, also it will include plenty duplicate values. The `host_neighbourhood` is dropped as it contains duplicate information compare to the `host_location` column. 

In [20]:
# Remove 'host_name','host_total_listings_count' and 'host_listings_count' columns
df_listings_clean.drop(['host_name','host_neighbourhood', 'host_total_listings_count', 'host_listings_count'],
                       inplace=True, axis=1)

We will also remove `host_total_listings_count` and `host_listings_count` as the `calculated_host_listings_count` is the correct column showing the number of listings per host after checking. 

In [21]:
# Create an array contains number of listings per host
host_listing_num=df_listings_clean.groupby('host_id').count()['id']

# Check if the number of listings per host matches the 'calculated_host_listings_count' column
for i in host_listing_num.index:
    if host_listing_num[i] != df_listings_clean[df_listings_clean['host_id']==i]['calculated_host_listings_count'].unique():
        print(i, 'This one is wrong')

192586 This one is wrong
236828 This one is wrong
253850 This one is wrong
273097 This one is wrong
299509 This one is wrong
646220 This one is wrong
851172 This one is wrong
1111601 This one is wrong
1150765 This one is wrong
1267171 This one is wrong
1915325 This one is wrong
2529624 This one is wrong
2711564 This one is wrong
2970569 This one is wrong
3008499 This one is wrong
3092851 This one is wrong
3503210 This one is wrong
3535509 This one is wrong
3797020 This one is wrong
4265889 This one is wrong
4295122 This one is wrong
5021059 This one is wrong
5351643 This one is wrong
5505419 This one is wrong
7324873 This one is wrong
7371942 This one is wrong
7768900 This one is wrong
7848001 This one is wrong
8047634 This one is wrong
8538394 This one is wrong
8601197 This one is wrong
9312244 This one is wrong
11335738 This one is wrong
11864182 This one is wrong
13071175 This one is wrong
13253365 This one is wrong
13262819 This one is wrong
13513326 This one is wrong
14630118 This

No error appears so the column `calculated_host_listings_count` matches `host_listing_count` column.

Next, we will look at the hot related columns that contain missing values.

In [22]:
# Create a list which has host related column names
host_columns_cleaned=[]
for i in df_listings_clean.columns:
    if 'host' in i:
        host_columns_cleaned.append(i)
        
# Create a series that contains number of null values in all remaing host related columns
host_cleaned_null=df_listings_clean[host_columns_cleaned].isnull().sum()

# Use list comprehension to show the columns that have null values
host_cleaned_null[[i for i in host_cleaned_null.index if host_cleaned_null[i] !=0]]

host_location            902
host_about              2386
host_response_time      1417
host_response_rate      1417
host_acceptance_rate     596
host_is_superhost         30
dtype: int64

The `host_location` column contains the location details about the host, the `host_about` column shows the host introductions. As these columns contains texts, we can fill the null values in these columns as 'None'.

In [23]:
# Fill null values in neighbourhood_overview columns
df_listings_clean[['host_location', 'host_about']] = df_listings_clean[['host_location', 'host_about']].fillna(value='None')

The `host_response_time`, `host_response_rate` and `host_acceptance_rate` reveals similar information about how active the hosts are regard to new bookings. We will deal with these three columns seperately by first looking at what the data is like.

In [24]:
# Check the values in 'host_response_time' columns
df_listings_clean['host_response_time'].value_counts()

within an hour        3595
within a few hours     574
within a day           265
a few days or more      59
Name: host_response_time, dtype: int64

In [25]:
# Check the values in 'host_response_rate' columns
df_listings_clean['host_response_rate'].value_counts()

100%    3648
99%      250
90%      134
98%       74
80%       32
0%        31
96%       28
97%       21
93%       20
92%       17
94%       17
50%       16
83%       16
67%       15
86%       14
81%       12
88%       12
60%       12
95%       11
84%       11
89%       11
75%       11
70%       10
71%        8
57%        8
91%        7
82%        6
63%        5
29%        5
20%        5
40%        4
25%        4
78%        3
66%        3
33%        2
30%        2
58%        2
34%        2
13%        1
87%        1
10%        1
43%        1
Name: host_response_rate, dtype: int64

In [26]:
# Check the values in 'host_acceptance_rate' columns
df_listings_clean['host_acceptance_rate'].value_counts()

100%    2099
99%      838
98%      271
97%      252
96%      178
        ... 
26%        1
13%        1
14%        1
30%        1
24%        1
Name: host_acceptance_rate, Length: 85, dtype: int64

Therefore, we will fill the null values in `host_response_time` as 'Not provided', convert the `host_response_rate` and `host_acceptance_rate` as floats from percentage string, and fill null values in these two columns as 0.

In [27]:
# Fill the null values in 'host_response_time' as 'Not provided'.
df_listings_clean['host_response_time'] = df_listings_clean['host_response_time'].fillna('Not provided')

# Convert the response rate as floats from percentage string and fill null values as 0.
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].apply(lambda col: float(str(col).replace('%', '')))
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].fillna(value=0)

# Convert the acceptance rate as floats from percentage string and fill null values as 0.
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].apply(lambda col: float(str(col).replace('%', '')))
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].fillna(value=0)

For the `host_is_superhost` column, the data shows if the host is a superhost using boolean values (t for true and f for false). Therefore, it is sensible to assume that a null value indicates that the host is not a superhost.

In [28]:
# Check the 'host_is_superhost' column.
df_listings_clean['host_is_superhost'].value_counts()

f    3322
t    2558
Name: host_is_superhost, dtype: int64

In [29]:
# Fillin null values for 'host_is_superhost' column by 'f'.
df_listings_clean['host_is_superhost'] = df_listings_clean['host_is_superhost'].fillna(value='f')

We can check if the current host related columns contain any null values

In [30]:
df_listings_clean[host_columns_cleaned].isnull().sum()

host_id                                         0
host_since                                      0
host_location                                   0
host_about                                      0
host_response_time                              0
host_response_rate                              0
host_acceptance_rate                            0
host_is_superhost                               0
host_verifications                              0
host_has_profile_pic                            0
host_identity_verified                          0
calculated_host_listings_count                  0
calculated_host_listings_count_entire_homes     0
calculated_host_listings_count_private_rooms    0
calculated_host_listings_count_shared_rooms     0
dtype: int64

### Clean location related columns

From the listing column names, we can extract the columns that contain property location informtion. The columns are `neighbourhood_overview`, `neighbourhood`, `neighbourhood_cleansed`,`latitude`, `longtitude`. We can first see what these columns contain, then check if these columns contain any null values.

In [31]:
# Check the location columns in the first row 
df_listings_clean[['neighborhood_overview', 'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude']].head(1).T

,0
neighborhood_overview,"The neighbourhood is in the historic New Town,..."
neighbourhood,"Edinburgh, City of Edinburgh, United Kingdom"
neighbourhood_cleansed,"Old Town, Princes Street and Leith Street"
latitude,55.95759
longitude,-3.18805


In [32]:
# Check if the property location related columns have any null values
df_listings_clean[['neighborhood_overview', 'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude']].isnull().sum()

neighborhood_overview     1589
neighbourhood             1589
neighbourhood_cleansed       0
latitude                     0
longitude                    0
dtype: int64

The `neighborhood_overview` column shows the neighborhood information hosts write about the listing, thus we can fill the null values in this column as 'None'. The `neighbourhood` column contains replicated information as the `neighborhood_overview` column, thus we can drop this column.

In [33]:
# Fill the null values in the neighborhood_overview column as None
df_listings_clean['neighborhood_overview'] = df_listings_clean['neighborhood_overview'].fillna('None')

# Remove neighbourhood column
df_listings_clean.drop('neighbourhood', axis=1, inplace= True)

### Clean property related columns

The listing dataset contains multiple columns that are related to the property itself. This includes `name`,`property_type`,`room_type`,`accomodates`,`bathrooms_text`,`beds`,and `amenities`. We will first see what these columns contain, then check if these columns contain null values. 

In [34]:
# Check the property related columns in the first row 
df_listings_clean[['name','property_type','room_type','accommodates','bathrooms_text','beds','amenities']].head(1).T

,0
name,Rental unit in Edinburgh · ★4.98 · 1 bedroom ·...
property_type,Entire rental unit
room_type,Entire home/apt
accommodates,2
bathrooms_text,1 bath
beds,1.0
amenities,[]


In [35]:
# Check if the property related columns contain any null values
df_listings_clean[['name','property_type','room_type','accommodates','bathrooms_text','beds','amenities']].isnull().sum()

name               0
property_type      0
room_type          0
accommodates       0
bathrooms_text     4
beds              47
amenities          0
dtype: int64

By checking the amentities column, we observe that no useful information is contained therefore we will drop this column.

In [36]:
# Check the amentities column
df_listings_clean['amenities'].value_counts()

[]    5910
Name: amenities, dtype: int64

In [37]:
# Drop amentities column
df_listings_clean.drop('amenities', axis=1, inplace=True)

The name of the property contains the information of the property_type, location, star rating, number of bedrooms and number of bathrooms. Therefore, we can extract the information in this column to fill the null values in the `bathrooms_text` and the `beds` columns first, then drop this column.

For the 'bathrooms_text', we can check what the rows that contain these data looks like in our dataframe.

In [38]:
# Check the rows that contains missing bathroom_text data.
df_listings_clean[df_listings_clean['bathrooms_text'].isnull()].T

,83,151,228,536
id,641460,1360543,2486691,6909721
name,Serviced apartment in Edinburgh · ★4.54 · 2 be...,Home in Edinburgh · ★4.67 · 1 bedroom · 1 bed,Rental unit in Edinburgh · ★4.93 · 1 bedroom ·...,Hostel in Edinburgh · ★4.68 · Studio · 1 bed
neighborhood_overview,There is a 24-hour Asda supermarket located wi...,None,"Beautiful neighbourhood, situated in a wide, t...",Whether you're visiting for the Fringe festiva...
host_id,2989965,7385507,12724800,20509940
host_since,2012-07-19,2013-07-09,2014-03-02,2014-08-25
host_location,"Edinburgh, United Kingdom","Edinburgh, United Kingdom","Edinburgh, United Kingdom",None
host_about,"Hi, We are Ocean Serviced Apartments.\r\nWe ha...","Local Scotsman, from Oban originally, but live...","Everyone has a story, let's talk!",None
host_response_time,Not provided,within a few hours,Not provided,within an hour
host_response_rate,0.0,100.0,0.0,96.0
host_acceptance_rate,100.0,0.0,0.0,100.0


In [39]:
# Check the distinct column values in the bathrooms_text column.
df_listings_clean['bathrooms_text'].value_counts()

1 bath               2887
1 shared bath         861
2 baths               803
1.5 baths             429
1 private bath        415
1.5 shared baths      165
3 baths                99
2.5 baths              78
2 shared baths         47
3.5 baths              21
0 baths                19
2.5 shared baths       15
4 baths                15
Half-bath              12
Shared half-bath       11
0 shared baths          7
5 baths                 6
5.5 baths               4
3 shared baths          3
4.5 baths               2
7 baths                 1
9 shared baths          1
9 baths                 1
6.5 baths               1
8.5 baths               1
6 baths                 1
Private half-bath       1
Name: bathrooms_text, dtype: int64

Therefore, we will first fill the null values as 0 for both newly added columns, then convert this text column into numerical columns by appending two numerical columns `bathroom_num` and `private_bath` where `bathroom_num` gives the number of bathrooms and `private_bath` shows 1 if the bathrooms are private, and 0 if the bathrooms are shared.

In [40]:
# Fill in the missing bathroom_text data
df_listings_clean['bathrooms_text']=df_listings_clean['bathrooms_text'].fillna(value='0 shared baths')

In [41]:
# Add 0.5 in front of Half baths
for i in range(df_listings_clean.shape[0]):
    if 'alf' in df_listings_clean['bathrooms_text'][i]:
        df_listings_clean.loc[i, 'bathrooms_text'] = '0.5 '+ df_listings_clean.loc[i, 'bathrooms_text']

In [42]:
# Extract numbers and if shared information from the bathroom text column
private_bath=[]
for i in range(df_listings_clean.shape[0]):
    if 'hared' in df_listings_clean.loc[i, 'bathrooms_text']:
        private_bath.append(0)
    else:
        private_bath.append(1)
    df_listings_clean.loc[i, 'bathrooms_text'] = float(df_listings_clean.loc[i, 'bathrooms_text'].split()[0])

# Add the binary column to our dataframe
df_listings_clean['private_bath']=private_bath

# Rename bathrooms_text to bathroom_num
df_listings_clean.rename(columns={'bathrooms_text': 'bathroom_num'}, inplace=True)

# Convert the column type to float
df_listings_clean['bathroom_num']=df_listings_clean['bathroom_num'].astype(float)

Now we can add the missing bedroom numbers by checking the name of the property.

In [43]:
# Get the row index for the rows contain missing beds data
index_beds_missing=df_listings_clean[df_listings_clean['beds'].isnull()].index

# Get the corresponding names data
names_beds_missing=df_listings_clean.loc[index_beds_missing, 'name']

# Extract bed counts from the names data
extract_beds=[]
for i in names_beds_missing.index:
    j=names_beds_missing[i].split()
    for k in range(len(j)):
        if 'bedroom' in j[k]:
            extract_beds.append(float(j[k-1]))
        elif 'Studio' in j[k]:
            extract_beds.append(1)
        elif 'Camper' in j[k]:
            extract_beds.append(1)

# Fill missing values of beds using extracted beds number
df_listings_clean.loc[df_listings_clean['beds'].isnull(), 'beds']=extract_beds

Then we can drop the `name` column

In [44]:
# Drop name column 
df_listings_clean.drop('name', axis=1, inplace=True)

### Clean other columns

We can check what are the columns that still contain null values in our current listing dataset.

In [45]:
# Create temporary series object that contains the number of null values for each column in the listing dataframe
temp_null_series=df_listings_clean.isnull().sum()

# Use list comprehension to show the columns that have null values
temp_null_series[[i for i in temp_null_series.index if temp_null_series[i] !=0]]

price               1216
has_availability    1216
license             5247
dtype: int64

By checking the percentage of the null values in the license column, we observe that it contains null values in nearly 90% of the rows. Therefore, we will drop the `license` column.

In [46]:
# Check the proportion of the null values in the license column
print('Percentage of Null Values % in the license column is')
print(round(df_listings_clean['license'].isnull().sum()/len(df_listings_clean)*100, 2)) #coverted into a percent(%) and round to 2 decimal places

Percentage of Null Values % in the license column is
88.78


In [47]:
# Drop the license column
df_listings_clean.drop('license', axis=1, inplace=True)

Next, we will check what values the `price` and `has_availability` columns contain.

In [48]:
# Check the price column
df_listings_clean['price'].value_counts()

$100.00      85
$150.00      76
$120.00      62
$90.00       61
$200.00      60
             ..
$995.00       1
$1,747.00     1
$545.00       1
$524.00       1
$407.00       1
Name: price, Length: 520, dtype: int64

Therefore, we can first convert the values into floats, then try to fill the null values by using other columns in the dataset to predict the price values.

In [49]:
# Convert 'price' column to numbers
price_int=[]
for i in range(len(df_listings_clean['price'])):
    p=df_listings_clean['price'][i]
    if type(p) == str:
        n=p.split('$')[1]
        if ',' in list(n):
            n_no_comma=''.join([x for x in n if x != ','])
            price_int.append(float(n_no_comma))  
        else:
            price_int.append(float(n))
    else:
        price_int.append(np.nan)
df_listings_clean['price']=price_int

# Convert existing column to float for future filtering
df_listings_clean['price']=df_listings_clean['price'].astype(float)

In [50]:
# Check the had_avalibility column
df_listings_clean['has_availability'].value_counts()

t    4684
f      10
Name: has_availability, dtype: int64

We will assume that the listings with this column absent has no availability, thus we will fill the null values using 'f'.

In [51]:
# Fill the null values in has_availability using 'f'
df_listings_clean['has_availability']=df_listings_clean['has_availability'].fillna(value='f')

### Check current null values 

In [132]:
# Return pandas series contain percentage of null values
per_null_listing=df_listings_clean.isnull().sum()/df_listings_clean.shape[0]

# Return the non null column name
non_null_col = per_null_listing.loc[per_null_listing!=0].index

# Return the null value percentage
per_null = per_null_listing.loc[per_null_listing!=0].values

print('The column that still contains null values is')
print(list(non_null_col)[0])
print('has percentage of null values %')
print(f'{round(float(per_null)*100, 2)}%')

The column that still contains null values is
price
has percentage of null values %
20.58%


In [58]:
# Check the percentage of the null values in the current review dataset
df_reviews_clean.isnull().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

We see that only `price` column has null values, so our next step is to use a simple linear regression to attempt filling these null values.

### Fillin null values in `price` column

#### Import libraries for linear regression

In [90]:
# Import linear regression libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#### Convert binary columns with 't' and 'f' to 0 and 1

In [103]:
# Select the binary columns then convert the values to 0 and 1 using a map function
for i in df_listings_clean.select_dtypes(include=['object']).columns:
    if set(df_listings_clean[i].unique()) == {'t', 'f'}:
        df_listings_clean[i] = df_listings_clean[i].map({'t': 0, 'f': 1})

#### Extract rows with null price values and select numerical columns

In [121]:
# Select numerical columns
df_listings_num=df_listings_clean.select_dtypes(exclude='object')

# Drop id columns
df_listings_num.drop(['id', 'host_id'], axis=1, inplace=True)

# Split the listing dataset into data with known price and rows with null values for price
df_price=df_listings_num[df_listings_num['price'].isnull()==False]
df_non_price=df_listings_num[df_listings_num['price'].isnull()]

#### Train test split on the labelled dataset

In [122]:
# Define features and target variable
X=df_price.drop('price', axis=1)
y=df_price['price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Scale the features

In [123]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Train the linear regression model

In [124]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [125]:
X_train

,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathroom_num,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,private_bath
761,90.0,100.0,1,0,0,55.956130,-3.164320,4,1.0,2.0,2,1125,2,2,1125,1125,2.0,1125.0,0,0,0,0,0,220,35,5,4.48,4.66,4.66,4.70,4.68,4.65,4.50,0,1,1,0,0,2.27,1
4921,99.0,94.0,1,0,0,55.955168,-3.155685,5,1.0,3.0,1,365,1,2,731,731,1.4,731.0,0,11,33,57,240,6,5,0,4.67,5.00,5.00,5.00,4.33,4.83,4.50,0,23,23,0,0,0.47,1
3222,0.0,100.0,0,0,0,55.962220,-3.196670,4,1.0,2.0,3,40,2,3,1125,1125,3.0,1125.0,0,6,33,52,98,141,51,2,4.89,4.94,4.89,4.94,4.95,4.86,4.87,0,2,2,0,0,3.57,1
4798,100.0,99.0,0,0,0,55.949093,-3.189248,4,1.0,2.0,2,7,1,2,1125,1125,2.0,1125.0,0,0,0,0,0,61,52,3,4.94,4.97,4.97,4.89,4.97,4.92,4.94,0,2,2,0,0,4.14,1
5287,100.0,100.0,0,0,0,55.949205,-3.213850,4,1.0,2.0,2,45,1,2,1125,1125,2.0,1125.0,0,15,37,61,137,8,8,2,4.88,5.00,5.00,4.88,4.88,4.88,4.88,0,23,23,0,0,1.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5605,100.0,100.0,0,0,0,55.966503,-3.165247,4,1.0,2.0,3,730,5,5,1125,1125,5.0,1125.0,0,11,11,11,40,4,4,0,4.75,5.00,5.00,5.00,5.00,4.75,4.75,0,2,1,1,0,0.94,1
559,90.0,100.0,0,0,0,55.946480,-3.191020,4,1.5,2.0,1,6,1,3,1125,1125,1.2,1125.0,0,0,0,0,0,135,53,6,4.91,4.93,4.90,4.94,4.94,4.96,4.82,0,1,1,0,0,1.32,1
3969,100.0,95.0,0,0,0,55.958680,-3.185580,2,1.5,1.0,1,5,1,1,5,5,1.0,5.0,0,22,51,80,161,75,45,3,4.91,4.92,4.85,4.92,4.93,4.91,4.57,1,1,0,1,0,3.69,1
4846,80.0,92.0,0,0,0,55.961240,-3.276270,2,1.0,1.0,1,365,1,1,365,365,1.0,365.0,0,27,57,87,358,31,26,0,4.74,4.77,4.68,4.87,4.77,4.81,4.74,1,3,0,3,0,2.10,0


In [126]:
r2_score(y_train, model.predict(X_train_scaled))

0.08533278875095573

In [127]:
y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)

R-squared Score: 0.04237663559883098


In [128]:
model.predict(df_non_price)

array([ 2.65489180e+02,  3.40526552e+01,  4.32282688e+02,  1.02870375e+02,
        7.93719392e+01,  2.12249875e+02,  3.90893815e+02,  1.31579847e+02,
        3.05775174e+02,  7.24919995e+02,  1.87978398e+02,  1.25506038e+02,
        1.93770805e+02,  2.91668632e+02,  6.89311400e+01, -1.17511644e+02,
        1.06998719e+02,  5.84854432e+01,  5.07486624e+02,  3.27995633e+02,
        2.12920563e+02,  6.14054035e+01,  1.43360986e+02,  2.79303599e+02,
        3.09799244e+02,  2.00696092e+02,  1.50749598e+02,  2.22339434e+02,
        1.43253653e+02,  2.48055503e+02,  1.92512758e+02,  1.61304978e+02,
        1.74142306e+02,  4.22590199e+02,  1.21263205e+02,  2.14882034e+02,
        3.38091587e+01,  5.32305099e+02,  2.81839698e+02,  2.53137277e+02,
        4.49202107e+02,  2.05430180e+02,  1.67031335e+02,  2.70871665e+02,
        4.33975221e+02,  2.32088353e+02,  7.21811725e+01,  1.49539181e+02,
        8.31142893e+01,  1.37658309e+02,  6.02314746e+01,  1.02375746e+02,
        1.62341369e+02,  